In [ ]:
import os
import subprocess as sp
import re
import requests

import googlesearch as gs
from bs4 import BeautifulSoup
from openai import OpenAI


openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# ReAct (Reason + Act) Framework (Yao et al, 2022)
- Thought
- Action
- Pause
- Observation
- Answer

In [ ]:
system_prompt = """
You run in a loop of Thought, Action, Pause, Observation.
At the end of the loop you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return Pause.
Observation will be the result of running those actions.

Your available actions are:
ping:
e.g. ping: python.org
Does a ping command and return the response time in seconds

bash:
e.g. bash: python --version
Returns the result of bash command execution

web_search:
e.g. web_search: capital of Portugal
Returns the content of the first result of a google search

Example session:
Question: How many islands make up Madeira?
Thought: I should do a web search for the Madeira
Action: web_search: Madeira
Pause

You will be called again with this:
Observation: Madeira is a Portuguese island chain made up of four islands: Madeira, Porto Santo, Desertas, and Selvagens, only two of which are inhabited (Madeira and Porto Santo.) 

You then output:
Answer: Four islands
"""

In [ ]:
def query_model(messages, model="gpt-4-turbo"):
    response = openai_client.chat.completions.create(
        model=model,
        messages=messages
    )
    return response.choices[0].message.content

In [ ]:
def ping(website: str):
    if not website.startswith("https://"):
        website = "https://" + website
    response = requests.get(website)
    return response.elapsed.total_seconds()


def bash(command: str):
    out = sp.check_output(command, shell=True)
    return out.decode("utf-8").rstrip()


def web_search(query: str):
    top_result = next(gs.search(query, advanced=True))
    print("Using data from: ", top_result.url)
    content = requests.get(top_result.url)
    return re.sub(" {2,}", "", BeautifulSoup(content.text, "lxml").text.replace("\n", "").replace("\r", ""))

In [ ]:
ping("2024.pycon.pt")

In [ ]:
bash("python --version")

In [ ]:
web_search("How many islands make up Madeira?")

In [ ]:
known_actions = {
    "ping": ping,
    "bash": bash,
    "web_search": web_search
}

In [ ]:
def query(user_query: str, max_iter: int = 5):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    counter = 1
    while counter < max_iter:
        print(f"Loop: {counter}")
        response = query_model(messages)
        print(response)
        actions = [re.match(r"^Action: (\w+): (.*)", a) for a in response.split('\n') if re.match(r"^Action: (\w+): (.*)", a)]
        if actions:
            action, action_input = actions[0].groups()
            print("Running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
            messages.append({"role": "user", "content": next_prompt})
        else:
             break
        counter += 1

In [ ]:
user_prompt = "What's the response time for google.com?"
query(user_prompt)

In [ ]:
user_prompt = "What python packages do I have installed?"
query(user_prompt)

In [ ]:
user_prompt = "What's new in python 3.14?"
query(user_prompt)

In [ ]:
user_prompt = "Who are the speakers at PyCon Portugal 2024?"
query(user_prompt)